In [2]:
from keras.api.applications import ResNet50
from keras.api.layers import Dense, Dropout, Flatten
from keras.api.models import Model
from keras.api.optimizers import Adam
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator

In [ ]:
input_shape = (224, 224, 3)
batch_size = 32
num_classes = 7
epochs = 20
learning_rate = 0.0001

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/media/gui/EA0CFB590CFB1F6F/repositories/FR-EAGI/datasets/FER2013/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/media/gui/EA0CFB590CFB1F6F/repositories/FR-EAGI/datasets/FER2013/test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=test_generator.samples // batch_size
)

model.save('modelo_emocoes_resnet50.h5')

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")